In [ ]:
import openpyxl

# File path
file_path = "/home/dan_pham/Downloads/Distraction/Nhập dữ liệu khảo sát.xlsx"

# Load the workbook and select the first worksheet
wb = openpyxl.load_workbook(file_path)
ws = wb.active  # or specify by name, e.g. wb["Sheet1"]

# Define columns to change (A=1, B=2, ..., so G=7, H=8, etc.)
columns = ["G", "H", "I", "P", "R", "X", "AE", "AF", "AI", "AK", "AM", "AQ", "AS", "AT", "AU", "AZ"]

# Define mapping rule
mapping = {"1": 5, "2": 4, "3": 3, "4": 2, "5": 1, 1: 5, 2: 4, 3: 3, 4: 2, 5: 1}

# Loop through rows 3–39
for row in range(3, 40):
    for col in columns:
        cell = ws[f"{col}{row}"]
        if cell.value in mapping:
            cell.value = mapping[cell.value]

# Save the modified file (to avoid overwriting original, use a new file name)
output_path = "/home/dan_pham/Downloads/Distraction/Nhập dữ liệu khảo sát_modified.xlsx"
wb.save(output_path)

print("✅ File updated and saved as:", output_path)


✅ File updated and saved as: /home/dan_pham/Downloads/Distraction/Nhập dữ liệu khảo sát_modified.xlsx


## CRONBACH'S ALPHA CALCULATION FOR SURVEY DATA

In [ ]:
# Load required packages
library(readxl)
library(psych)

# === 1. Read the Excel file ===
file_path <- "/home/dan_pham/Downloads/Distraction/Nhập dữ liệu khảo sát_modified.xlsx"

# Read the full dataset (assumes first row contains headers)
df <- read_excel(file_path)

# Skip the first row if it contains question labels instead of data
df_data <- df[-1, ]

# === 2. Select the two groups of questions ===
# Adjust column ranges if needed based on Excel layout
job_related <- df_data[, 7:31]   # Columns G to AE
habit_related <- df_data[, 32:52]  # Columns AF to AZ

# Convert all columns to numeric (ignore non-numeric entries)
job_related <- data.frame(lapply(job_related, function(x) as.numeric(as.character(x))))
habit_related <- data.frame(lapply(habit_related, function(x) as.numeric(as.character(x))))

# === 3. Function to compute reliability report ===
compute_alpha_report <- function(df_section, section_name) {
  df_clean <- na.omit(df_section)
  
  cat("\n==============================\n")
  cat("Section:", section_name, "\n")
  cat("==============================\n")
  
  # Case Processing Summary
  cat("\nCase Processing Summary (Listwise deletion based on all variables in the procedure):\n")
  cat("Total Cases:", nrow(df_section), "\n")
  cat("Valid (listwise):", nrow(df_clean), "\n")
  
  # Reliability Statistics
  alpha_result <- psych::alpha(df_clean)
  cat("\nReliability Statistics:\n")
  cat("Cronbach's Alpha:", round(alpha_result$total$raw_alpha, 3), "\n")
  cat("Number of Items:", ncol(df_clean), "\n")
  
  # Item-Total Statistics (robust to psych version differences)
  cat("\nItem-Total Statistics:\n")
  item_stats <- alpha_result$item.stats
  possible_cols <- c("r.drop", "r.cor", "alpha.if.deleted", "alpha.drop")
  cols_present <- intersect(possible_cols, colnames(item_stats))
  print(item_stats[, cols_present, drop = FALSE])
}


In [ ]:
# ==== Load libraries ====
library(readxl)
library(psych)
library(dplyr)

# ==== Step 1: Load your dataset ====
file_path <- "Nhập dữ liệu khảo sát_modified.xlsx"
data <- read_excel(file_path)

# ==== Step 2: Define question sections ====
job_questions   <- data[, 7:31]   # G to AE: CÁC CÂU HỎI LIÊN QUAN CÔNG VIỆC
habit_questions <- data[, 32:52]  # AF to AZ: CÂU HỎI VỀ THÓI QUEN

# ==== Step 3: Function to convert non-numeric to numeric ====
convert_to_numeric <- function(df) {
  df[] <- lapply(df, function(x) {
    if (is.numeric(x)) return(x)
    as.numeric(as.factor(trimws(as.character(x))))
  })
  return(df)
}

# ==== Step 4: Function to compute Cronbach’s alpha and print SPSS-style output ====
analyze_alpha <- function(df, label) {
  df <- convert_to_numeric(df)
  df <- na.omit(df)

  if (ncol(df) < 2) {
    cat("⚠️ Not enough variables in", label, "to compute alpha\n")
    return(NULL)
  }

  # Compute Cronbach’s alpha
  alpha_result <- psych::alpha(df, warnings = FALSE, check.keys = TRUE)

  # === Case Processing Summary ===
  case_summary <- paste0(
    "===== Case Processing Summary (Listwise deletion based on all variables in the procedure) =====\n",
    "Number of valid cases: ", nrow(df), "\n",
    "Number of items: ", ncol(df), "\n\n"
  )

  # === Reliability Statistics ===
  reliability_stats <- capture.output({
    cat("===== Reliability Statistics =====\n")
    rs <- data.frame(
      `Cronbach's Alpha` = round(alpha_result$total$raw_alpha, 3),
      `Cronbach's Alpha Based on Standardized Items` = round(alpha_result$total$std.alpha, 3),
      `N of Items` = ncol(df)
    )
    print(rs, row.names = FALSE)
  })

  # === Item-Total Statistics ===
  item_stats <- alpha_result$item.stats
  available_cols <- intersect(colnames(item_stats),
                              c("sd", "r.drop", "r.cor", "r.smc", "alpha.if.deleted"))
  item_total_stats <- capture.output({
    cat("\n===== Item-Total Statistics =====\n")
    stats_df <- item_stats[, available_cols, drop = FALSE]
    colnames(stats_df) <- c(
      "Scale Variance if Item Deleted",
      "Corrected Item-Total Correlation",
      "Squared Multiple Correlation",
      "Cronbach's Alpha if Item Deleted"
    )[1:length(available_cols)]
    print(round(stats_df, 3))
  })

  # === Combine and write to file ===
  full_output <- c(case_summary, reliability_stats, "\n", item_total_stats)
  output_file <- paste0(label, "_Cronbach_results.txt")
  writeLines(full_output, output_file)

  cat("✅ Results saved to:", output_file, "\n")
}

# ==== Step 5: Run analysis for both sections ====
analyze_alpha(job_questions, "job_questions")
analyze_alpha(habit_questions, "habit_questions")


## CORRECTED ITEM-TOTAL CORRELATION

In [ ]:
# Load libraries
library(readxl)
library(psych)
library(dplyr)

# === Step 1: Load data ===
file_path <- "/home/dan_pham/Downloads/Distraction/Nhập dữ liệu khảo sát_modified.xlsx"
data <- read_excel(file_path)

# === Step 2: Define two question sets ===
job_questions <- data[, 7:31]    # G:AE — CÁC CÂU HỎI LIÊN QUAN CÔNG VIỆC
habit_questions <- data[, 32:52] # AF:AZ — CÂU HỎI VỀ THÓI QUEN

# === Step 3: Convert all text/factor columns to numeric ===
convert_to_numeric <- function(df) {
  df[] <- lapply(df, function(x) {
    if (is.numeric(x)) return(x)
    as.numeric(as.factor(trimws(as.character(x))))
  })
  return(df)
}

# === Step 4: Main function to calculate Cronbach's alpha ===
analyze_alpha <- function(df, label) {
  df <- convert_to_numeric(df)
  df <- na.omit(df)

  if (ncol(df) < 2) {
    cat("Not enough variables in", label, "to compute alpha\n")
    return(NULL)
  }

  alpha_result <- psych::alpha(df, warnings = FALSE, check.keys = TRUE)

  # Case Processing Summary
  summary_text <- paste0(
    "===== Case Processing Summary (Listwise deletion based on all variables) =====\n",
    "Number of valid cases: ", nrow(df), "\n\n"
  )

  # Reliability Statistics
  reliability_text <- capture.output({
    cat("===== Reliability Statistics for", label, "=====\n\n")
    print(alpha_result$total)
  })

  # Item-Total Statistics
  item_stats <- alpha_result$item.stats
  available_cols <- intersect(colnames(item_stats),
                              c("r.drop", "r.cor", "alpha.if.deleted", "corrected.item.total", "item.total", "raw.alpha"))
  if (length(available_cols) == 0) {
    item_total_text <- "\n(No matching item-total correlation columns found)\n"
  } else {
    item_total_text <- capture.output({
      cat("\n===== Item-Total Statistics for", label, "=====\n\n")
      print(item_stats[, available_cols, drop = FALSE])
    })
  }

  # Combine all
  full_output <- c(summary_text, reliability_text, "\n", item_total_text)

  # Write to file
  output_file <- paste0(label, "_results.txt")
  writeLines(full_output, output_file)
  cat("Results saved to:", output_file, "\n")
}

# === Step 5: Run for both datasets ===
analyze_alpha(job_questions, "job_questions")
analyze_alpha(habit_questions, "habit_questions")
